### This is an attempt to create a model, using synthetic data, to show how the flow of waste through a pipe can be monitored to predict when it will clog.  We will only use 2 features:

### Our X-axis will be the percentage of the amount of waste that can go through the pipe.  Each pipe will start at 100%, let's imagine that this particular size pipe has a maximum of 50 gallons per minute that can flow through it.  As we more forward in time, let's say 200 days, the pipe may only be at 25 gallons per minute, or 50%.  As a precaution, we'd like to focus on pipes that go under 20% of maximum allowable flow so these can be cleaned before they clog and cause an SSO.

### Our Y-axis, and y-variable, will be the number of days.  We will predict the number of days that will occur before a particular pipe is expected to dip below 20% of its maximum flow.

### Let's first create a few dataframes.  Each will be different sizes to gauge how well the models can predict.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import proof_of_concept_helpers
from proof_of_concept_helpers import create_pipe_data

# Exploring
import scipy.stats as stats

# Modeling
import statsmodels.api as sm

from scipy.stats import pearsonr

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error

In [ ]:
pipe_1 = create_pipe_data(105)

In [ ]:
pipe_1.head()

In [ ]:
pipe_1.tail()

In [ ]:
plt.scatter(pipe_1.percent_flow, pipe_1.days)

In [ ]:
pipe_1.shape

In [ ]:
int(pipe_1.shape[0] * .7)

In [ ]:
poly_regression(pipe_1, .7)

In [ ]:
# train = pipe_1[:70]

In [ ]:
# train.head()

In [ ]:
# train.tail()

In [ ]:
# X_train = train[['percent_flow']]
# y_train = train[['days']]

In [ ]:
# test = pipe_1[70:]

In [ ]:
# test.head()

In [ ]:
# test.tail()

### Linear Regression

In [ ]:
# X_test = test[['percent_flow']]
# y_test = test[['days']]

In [ ]:
# lm1 = LinearRegression()
# lm1.fit(X_train, y_train)

# lm1_y_intercept = lm1.intercept_
# print('Y-intercept: ' + str(lm1_y_intercept))

# lm1_coefficients = lm1.coef_
# print('Coefficients: ' + str(lm1_coefficients))

# y_pred_lm1 = lm1.predict(X_train)

In [ ]:
# mse_lm1 = mean_squared_error(y_train, y_pred_lm1)
# print("lm1\n  mse: {:.3}".format(mse_lm1)) 

In [ ]:
# r2_lm1 = r2_score(y_train, y_pred_lm1)

# print('  {:.2%} of the variance in the pipe''s flow can be explained by the increase in number of days.'.format(r2_lm1))

In [ ]:
# y_pred_test_lm1 = lm1.predict(X_test)
# mse_test_lm1 = mean_squared_error(y_test, y_pred_test_lm1)
# print("lm1\n  mse: {:.3}".format(mse_test_lm1)) 

In [ ]:
# r2_test_lm1 = r2_score(y_test, y_pred_test_lm1)

# print('  {:.2%} of the variance in the pipe''s flow can be explained by the increase in number of days.'.format(r2_test_lm1))

### Linear is not so great on the test.  Let's try polynomial 2 degrees after we take a peek at the results.

In [ ]:
# y_pred_lm1

In [ ]:
# y_pred_test_lm1

### Polynomial

In [ ]:
# poly_features = PolynomialFeatures(degree=2)

In [ ]:
# ### transforms the existing features to higher degree features.
# X_train_poly = poly_features.fit_transform(X_train)

# ### fit the transformed features to Linear Regression
# poly_model = LinearRegression()
# poly_model.fit(X_train_poly, y_train)
    
# ### predicting on training data-set
# y_train_predicted = poly_model.predict(X_train_poly)

In [ ]:
# rmse_train = np.sqrt(mean_squared_error(y_train, y_train_predicted))
# r2_train = r2_score(y_train, y_train_predicted)

In [ ]:
# print("The model performance for the training set")
# print("-------------------------------------------")
# print("RMSE of training set is {}".format(rmse_train))
# print("R2 score of training set is {}".format(r2_train))

In [ ]:
# y_test_predicted = poly_model.predict(poly_features.fit_transform(X_test))
# rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predicted))
# r2_test = r2_score(y_test, y_test_predicted)
# print("The model performance for the test set")
# print("-------------------------------------------")
# print("RMSE of test set is {}".format(rmse_test))
# print("R2 score of test set is {}".format(r2_test))

In [ ]:
# y_test_predicted

In [ ]:
# pipe_1_days = pd.DataFrame(pipe_1_days, columns=['days'])

In [ ]:
# y_test_predicted = pd.DataFrame(y_test_predicted, columns=['days'])

In [ ]:
# X_test.reset_index(drop=True, inplace=True)

In [ ]:
# pipe_1_results = X_test.merge(y_test_predicted, left_index=True, right_index=True)

In [ ]:
# pipe_1_results

In [ ]:
# pipe_1_results[pipe_1_results.percent_flow <= 20][:1]

In [ ]:
# y_train.iloc[-1][-1:]

In [ ]:
# print('This pipe is expected to reach 20% of maximum flow at day ' +str(pipe_1_results['days'][pipe_1_results.percent_flow <= 20][:1]))

In [ ]:
# print('Today is day ' + str(y_train.iloc[-1][-1:]))

In [ ]:
# pipe_1.tail()